# <font color='#2F4F4F'>1. Defining the Question</font>

## a) Specifying the Data Analysis Question

Conduct an evaluation to determine whether data science can help a supermarket chain to adhere to alcohol laws by making sure they do not sell alcohol to underage people

## b) Defining the Metric for Success
We will have succeeded if we can build a model for verifying people's age and evaluate it using Mean Absolute Error (MAE) 

## c) Understanding the context 
The supermarket chain would like to explore whether Data Science can help them adhere to
alcohol laws by making sure they do not sell alcohol to people underage. You are asked to
conduct that evaluation, so as you set to work, keep the following in mind:


1.   The shops are equipped with cameras in the checkout area which are triggered when a
person is buying alcohol.
2.   Computer vision methods can be used to determine the age of a person from a photo.
3. The task then is to build and evaluate a model for verifying people's age.

## d) Recording the Experimental Design

1. Import the Data
2. Perform exploratory data analysis to get an overall impression of the dataset.
3. Train and evaluate the model (it needs to be done on the GPU platform).
4. Combine your code, output, and findings (from the previous points) in the final notebook.
5. Make conclusions of the model evaluation, add them to the notebook.

## e) Data Relevance
The data was relevant for our analysis

In [1]:
# Importing Pre-requisites
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow import keras
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, Flatten, Dense, AveragePooling2D, GlobalAveragePooling2D
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet50
import tensorflow as tf

In [5]:
labels = pd.read_csv('/content/labels.csv')

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255)

In [3]:
!mkdir Images

In [ ]:
from zipfile import ZipFile

images = "/content/final_files (1).zip"
with ZipFile(images, 'r') as zip:
  zip.printdir()
  zip.extractall('Images')
  print('Done!')


In [16]:
train_generator = train_datagen.flow_from_dataframe(
dataframe=labels,
directory='/content/Images/final_files',
x_col="file_name",
y_col="real_age",
subset="training",
batch_size=32,
seed=12345,
shuffle=False,
class_mode="raw",
target_size=(100,100)
)

Found 7591 validated image filenames.


In [17]:
test_datagen = ImageDataGenerator(rescale=1./255,
                                  validation_split=0.2)
test_generator = test_datagen.flow_from_dataframe(
        dataframe=labels,
        directory='/content/Images/final_files',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        subset='validation',
        seed=12345)

Found 1518 validated image filenames.


In [18]:
optimizer = Adam(learning_rate = 0.0001)
backbone = ResNet50(input_shape=(224, 224, 3),
                        weights='imagenet', 
                        include_top=False)

model = Sequential()
model.add(backbone)
model.add(GlobalAveragePooling2D())
model.add(Dense(1, activation='relu'))

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])

94781440/94765736 [==============================] - 1s 0us/step


In [19]:
def train_model(model, train_generator, test_generator,batch_size=None,epochs=5, steps_per_epoch =None, validation_steps=None):
    my_callbacks = [ tf.keras.callbacks.EarlyStopping(patience=35)] 
    if steps_per_epoch is None:
        steps_per_epoch = len(train_generator)
    if validation_steps is None:
        validation_steps = len(test_generator)
    model.fit(train_generator,
          validation_data=test_generator,
          steps_per_epoch=steps_per_epoch,
          validation_steps=validation_steps,
          verbose=1, epochs=epochs,  callbacks=my_callbacks)
    return model

In [20]:
trained_model = train_model(model, train_generator, test_generator,batch_size=None,epochs=5, steps_per_epoch =None, validation_steps=None)

Epoch 1/5
238/238 [==============================] - 1841s 8s/step - loss: 256.4859 - mae: 11.6996 - val_loss: 385.3423 - val_mae: 14.6495
Epoch 2/5
238/238 [==============================] - 1850s 8s/step - loss: 73.8100 - mae: 6.5781 - val_loss: 586.0945 - val_mae: 19.0746
Epoch 3/5
238/238 [==============================] - 1762s 7s/step - loss: 38.4843 - mae: 4.7476 - val_loss: 346.6274 - val_mae: 13.9519
Epoch 4/5
238/238 [==============================] - 1793s 8s/step - loss: 23.3197 - mae: 3.7284 - val_loss: 329.6147 - val_mae: 13.8466
Epoch 5/5
238/238 [==============================] - 1798s 8s/step - loss: 15.5946 - mae: 3.0076 - val_loss: 335.4835 - val_mae: 14.2175


The process took too long per Epoch averaging 8s/step 